
# Prerequisite

In [1]:

#import required libraries
from azure.ml import MLClient
from azure.ml.entities import CommandJob, Code, PipelineJob, Dataset, InputDatasetEntry

In [2]:

#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [4]:
#get a handle to the workspace
ml_client = MLClient(subscription_id, resource_group, workspace)

# Pipeline job with command jobs
## Create command jobs

In [10]:
# define command jobs
environment = "AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:5"
parent_dir = '.'
train_job = CommandJob(
    inputs=dict(
        training_data=InputDatasetEntry(dataset=Dataset(local_path=parent_dir + "/data/")),
        max_epocs=20,
        learning_rate=1.8,
        learning_rate_schedule='time-based',
        ),
    outputs=dict(
        model_output=None
        ),
    display_name="my-train-job",
    code=Code(local_path=parent_dir + "/train_src"),
    environment=environment,
    compute="cpu-cluster",
    command="""python train.py --training_data ${{inputs.training_data}} --max_epocs ${{inputs.max_epocs}}
        --learning_rate ${{inputs.learning_rate}} --learning_rate_schedule 
        ${{inputs.learning_rate_schedule}} --model_output ${{outputs.model_output}}"""
    )

score_job = CommandJob(
    inputs=dict(
        model_input=InputDatasetEntry(dataset=Dataset(local_path=parent_dir + "/data/")),
        test_data=InputDatasetEntry(dataset=Dataset(local_path=parent_dir + "/data/")),
        ),
    outputs=dict(
        score_output=None
        ),
    display_name="my-score-job",
    code=Code(local_path=parent_dir + "/score_src"),
    environment=environment,
    command="""python score.py --model_input ${{inputs.model_input}}
        --test_data ${{inputs.test_data}} --score_output ${{outputs.score_output}}"""
        )

eval_job = CommandJob(
    inputs=dict(
        scoring_result=InputDatasetEntry(dataset=Dataset(local_path=parent_dir + "/data/")),
    ),
    outputs=dict(
        eval_output=None
    ),
    display_name="my-evaluate-job",
    environment=environment,
    command='echo "hello world"'
)

## Build pipeline

In [12]:
from azure.ml import dsl
from azure.ml.dsl import Pipeline
from azure.ml.entities import Dataset

def generate_dsl_pipeline() -> Pipeline:
    # 1. Construct pipeline with command job
    @dsl.pipeline(
        compute="cpu-cluster",
    )
    def sample_pipeline(
            pipeline_job_training_input,
            pipeline_job_test_input,
            pipeline_job_training_max_epocs,
            pipeline_job_training_learning_rate,
            pipeline_job_learning_rate_schedule,
    ):
        
        train_func = dsl.load_component(component=train_job)
        train_node = train_func(
            training_data=pipeline_job_training_input,
            max_epocs=pipeline_job_training_max_epocs,
            learning_rate=pipeline_job_training_learning_rate,
            learning_rate_schedule=pipeline_job_learning_rate_schedule,
        )

        score_func = dsl.load_component(component=score_job)
        score_node = score_func(
            model_input=train_node.outputs.model_output,
            test_data=pipeline_job_test_input,
        )
        # add env variable
        score_node.environment_variables = {"AZUREML_COMPUTE_USE_COMMON_RUNTIME": "false", "abc": "def"}
        eval_func = dsl.load_component(component=eval_job)
        eval_node = eval_func(scoring_result=score_node.outputs.score_output)
        return {
            "pipeline_job_trained_model": train_node.outputs.model_output,
            "pipeline_job_scored_data": score_node.outputs.score_output,
            "pipeline_job_evaluation_report": eval_node.outputs.eval_output,
        }

    pipeline = sample_pipeline(
        Dataset(local_path=parent_dir + "/data/"),
        Dataset(local_path=parent_dir + "/data/"),
        20,
        1.8,
        "time-based",
    )
    pipeline.outputs.pipeline_job_trained_model.mode = "upload"
    pipeline.outputs.pipeline_job_scored_data.mode = "upload"
    pipeline.outputs.pipeline_job_evaluation_report.mode = "upload"

    return pipeline

## Submit pipeline job

In [13]:
# create pipeline instance
pipeline = generate_dsl_pipeline()
# submit job to workspace
ml_client.jobs.create_or_update(pipeline, experiment_name="command_job_in_pipeline", continue_run_on_step_failure=True)

Uploading train_src:   0%|          | 0.00/3.31k [00:00<?, ?B/s]
Uploading COMPONENT_PLACEHOLDER: 100%|██████████| 41.0/41.0 [00:00<00:00, 178B/s]
compute is not a known attribute of class <class 'azure.ml._restclient.v2021_10_01.models._models_py3.PipelineJob'> and will be ignored


PipelineJob({'display_name': 'upbeat_melon_tl1mpxhz', 'type': 'pipeline', 'status': 'Preparing', 'log_files': None, 'parent_job_name': None, 'name': 'efff3541-9884-4db0-b405-ad17b47bbcaf', 'description': None, 'tags': {'azureml.pipelineComponent': 'pipelinerun'}, 'properties': {'mlflow.source.git.repoURL': 'https://github.com/cloga/azureml-examples.git', 'mlflow.source.git.branch': 'lochen/pipeline-env', 'mlflow.source.git.commit': 'bdc37ea05ca835191ccd9cf79217477308fd894c', 'azureml.git.dirty': 'True', 'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'MFE', 'runType': 'HTTP', 'azureml.parameters': '{"pipeline_job_training_max_epocs":"20","pipeline_job_training_learning_rate":"1.8","pipeline_job_learning_rate_schedule":"time-based"}', 'azureml.continue_on_step_failure': 'True', 'azureml.pipelineComponent': 'pipelinerun'}, 'id': '/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/user_studies/providers/Microsoft.MachineLearningServices/workspaces/CLIV1Pipelines/j